In [ ]:
# Learning Pytorch
# By writing a simple sentiment analyzer
# Uses 1 hot encoding

In [27]:
import os
import sys, csv
import torch
import torchtext
import pandas as pd
from torchtext.datasets import text_classification
from torchtext.utils import download_from_url, extract_archive

In [46]:
# constants
NGRAMS = 3
Train_Data_Size = 10000
Test_Data_Size = 1000

In [18]:
# hard code or else, OverFlowError in csv.field_size_limit
sys.maxsize = 2**16 - 1

In [22]:
# Source : https://pytorch.org/text/_modules/torchtext/datasets/text_classification.html
URLS = {
    'AmazonReviewPolarity':
        'https://drive.google.com/uc?export=download&id=0Bz8a_Dbh9QhbaW12WVVZS2drcnM'
}

In [25]:
# File sizes are too big, will use a chunk of it, train : 10000 rows, test 1000 rows
tar_data = download_from_url(URLS['AmazonReviewPolarity'])
extracted_files = extract_archive(tar_data)

In [28]:
extracted_files

['.data\\amazon_review_polarity_csv/test.csv',
 '.data\\amazon_review_polarity_csv/train.csv',
 '.data\\amazon_review_polarity_csv/readme.txt']

In [47]:
train_data = pd.read_csv('.data\\amazon_review_polarity_csv/train.csv', chunksize=1000).get_chunk(Train_Data_Size)
test_data = pd.read_csv('.data\\amazon_review_polarity_csv/test.csv', chunksize=1000).get_chunk(Test_Data_Size)

In [53]:
# Rename Columns
train_data.set_axis(['Sentiment', 'Title', 'Review'], axis=1, inplace=True)
test_data.set_axis(['Sentiment', 'Title', 'Review'], axis=1, inplace=True)